In [1]:
from pathlib import Path

folder = Path('XX01ZVNS2B')
target = folder / 'reference.pdb'
assert target.exists()

In [2]:
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from IPython.display import display, HTML

prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
prbo: ModuleType = pyrosetta.rosetta.basic.options
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring

logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                ex1=None,
                                                ex2=None,
                                                #mute='all',
                                                ignore_unrecognized_res=True,
                                                load_PDB_components=False,
                                                ignore_waters=True)
                               )

/Users/user/.conda/envs/py310/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [15]:
pyrosetta.rosetta.basic.options.set_boolean_option('run:ignore_zero_occupancy', False)
pyrosetta.rosetta.basic.options.set_boolean_option('in:auto_setup_metals', True)
pose: pyrosetta.Pose = pyrosetta.pose_from_file(str(target.absolute()))

In [16]:
from collections import Counter

con_tally: dict = Counter([type(con).__name__ for con in pose.constraint_set().get_all_constraints()]).most_common()

print('chains', prc.pose.conf2pdb_chain(pose))
print('sequence', pose.sequence())
print('Constraints present', con_tally)

chains map_unsigned_long_char{1: A, 2: B}
sequence DMYIERAGDITWEKDAEVTGNSPRLDVALDESGDFSLVEGETTDGVYRVMTRRLLGSTQVGVGVMQEGVFHTMWHVTKGAALRSGEGRLDPYWGDVKQDLVSYCGPWKLDAAWDGLSEVQLLAVPPGERAKNIQTLPGIFKTKDGDIGAVALDYPAGTSGSPILDKCGRVIGLYGNGVVIKNGSYVSAITQGKRE
Constraints present []


In [11]:
#pose: pyrosetta.Pose = pose.split_by_chain(1)

In [12]:
original = pose.clone()
#original: pyrosetta.Pose = pyrosetta.pose_from_file(str(target.absolute())).split_by_chain(1)

In [13]:
scorefxn: pr_scoring.ScoreFunction = pyrosetta.get_fa_scorefxn()
scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, 20)
scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, 5)
scorefxn.set_weight(pr_scoring.ScoreType.atom_pair_constraint, 5)
prbo.set_boolean_option('relax:constrain_relax_to_start_coords', True)
pyrosetta.rosetta.protocols.relax.FastRelax.register_options()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 15)
relax.constrain_relax_to_start_coords(True)
relax.apply(pose)
pose.dump_pdb('template.pdb')

True

In [14]:
prp.toolbox.CA_superimpose(original, pose)
print(prc.simple_metrics.metrics.RMSDMetric(original).calculate(pose))
pose.dump_pdb('template2.pdb')

1.071442963233211


True

In [8]:
con_tally: dict = Counter([type(con).__name__ for con in pose.constraint_set().get_all_constraints()]).most_common()

print('chains', prc.pose.conf2pdb_chain(pose))
print('sequence', pose.sequence())
print('Constraints present', con_tally)

chains map_unsigned_long_char{1: A, 2: z}
sequence DMYIERAGDITWEKDAEVTGNSPRLDVALDESGDFSLVEX
Constraints present [('CoordinateConstraint', 157)]
